## PropertyPilot Demo
## Conversational AI Capstone I
## 21 May 2024

In [108]:
# %pip list | grep google-cloud-aiplatform
# %pip list | grep google-api-core
# %pip install google-cloud-aiplatform==1.43.0
# %pip install google-api-core==2.17.1

In [109]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, ChatSession, Part
import vertexai.preview.generative_models as generative_models

import geopandas as gpd
from shapely.geometry import Point

import http.client
import json
import urllib.parse
from typing import Dict, Any
import pandas as pd
import folium

In [110]:
vertexai.init(project = "adsp-capstone-property-pilot", location = "us-central1")

## Model Setting - Gemini pro 1.0

In [111]:
model = GenerativeModel("gemini-1.0-pro")
chat = model.start_chat()

def get_chat_response(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=True)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

In [112]:
# Test chat
prompt = "Hello."
print(get_chat_response(chat, prompt))

Hello! I'm happy to hear from you. What would you like to talk about today?


## Settings

In [113]:
with open('/home/jupyter/zillow_api.txt', 'r') as file:
    zillow_api = file.read()

import requests

url = "https://zillow-com1.p.rapidapi.com/propertyExtendedSearch"

headers = {
	"X-RapidAPI-Key": zillow_api,
	"X-RapidAPI-Host": "zillow-com1.p.rapidapi.com"
}

with open('/home/jupyter/yelp_api.txt', 'r') as file:
    yelp_api = file.read()

yelp_api_key = yelp_api

## Import functions

In [114]:
import os

In [115]:
%run /home/jupyter/property_pilot/scripts/chat_response.py

In [116]:
%run /home/jupyter/property_pilot/scripts/generate_api_filter.py

In [117]:
%run /home/jupyter/property_pilot/scripts/fetch_property_info.py

In [118]:
%run /home/jupyter/property_pilot/scripts/yelp_property.py

In [119]:
%run /home/jupyter/property_pilot/scripts/neighborhood_rag.py

In [120]:
%run /home/jupyter/property_pilot/scripts/yelp_advisor.py

## Import data

In [121]:
# For RAG

# read csv
vector_store = pd.read_csv('/home/jupyter/property_pilot/data/vector_store.csv')
# embedding to numpy array
vector_store['embeddings'] = vector_store['embeddings'].apply(lambda x: np.array(list(map(float, x.split(',')))))

## Import prompts

In [122]:
os.chdir('/home/jupyter/property_pilot/prompts')

# prompt_classifier
file_path = 'instruction_classifier.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_classifier = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}") 
    
# prompt_apifilter
file_path = 'generate_prompt_apifilter.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_apifilter = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# yelp_categories
file_path = 'yelp_categories.txt'
try:
    with open(file_path, 'r') as file:
        yelp_categories = file.read().splitlines()
        categories_string = ','.join(yelp_categories)
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# prompt_property(for final output)
file_path = 'generate_prompt_property.txt'
try:
    with open(file_path, 'r') as file:
        instruction_prompt_property = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# instruction_rag_neighborhood
file_path = 'instruction_rag_neighborhood.txt'
try:
    with open(file_path, 'r') as file:
        instruction_rag_neighborhood = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# yelp_advisor_instructions (for yelp local advisor)
file_path = 'yelp_advisor_instructions.txt'
try:
    with open(file_path, 'r') as file:
        yelp_advisor_instruction = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")
    
# yelp_output_instructions (for yelp local advisor)
file_path = 'yelp_output_instructions.txt'
try:
    with open(file_path, 'r') as file:
        yelp_output_instructions = file.read()
except FileNotFoundError:
    print("The file does not exist")
except Exception as e:
    print(f"An error occurred: {e}")

## Property Plot main function

In [123]:
def pp_chatbot(user_query):
    
    # User's quety classify
    prompt_classifier = generate_prompt_classifier(instruction_prompt_classifier, user_query)
    response_int = int(get_chat_response(chat, prompt_classifier))
    
    # if the user is looking for a property     
    if response_int == 1:

        # get prompt for api filter 
        prompt_apifilter = generate_prompt_apifilter(instruction_prompt_apifilter, user_query)

        # get api filter
        apifilter = get_chat_response(chat, prompt_apifilter)
        apifilter = extract_json_to_dict(apifilter)

        # get listings from zillow
        response = requests.get(url, headers = headers, params = apifilter)

        # get top 3 listings(sorted by newest)
        # Define fields to extract
        fields = ["propertyType", "address", "price", "bedrooms", "bathrooms", "detailUrl", "imgSrc", "longitude", "latitude"]
        top_properties = extract_properties(response, 3, fields)

        # get description of the properties
        top_properties = fetch_descriptions(top_properties)

        # get resoFacts(detail) of the properties
        keys_to_fetch_resoFacts = [
            'hasGarage', 'hasPetsAllowed', 'heating', 'cooling', 'flooring', 'appliances',
            'laundryFeatures', 'associationFee',
            'livingArea', 'taxAnnualAmount', 'parkingFeatures', 'stories'
        ]
        top_properties = fetch_resoFacts(top_properties, headers, keys_to_fetch_resoFacts)

        # get school info of the properties
        top_properties = fetch_schools(top_properties, headers)
        
        # get restaurant info from yelp api
        yelp_results = fetch_top_businesses_near_properties(yelp_api_key, top_properties, user_query, yelp_categories, chat)
        
        # extract restaurant info
        yelp_fields = ['business_name', 'categories_titles', 'rating', 'latitude', 'longitude', 'display_address', 'url']
        top_restaurants = extract_business_info(yelp_results, 2, yelp_fields)
        
        # merge top_restaurants with top_properties
        top_properties = merge_property_and_restaurant_info(top_properties, top_restaurants)

        # neighborhood boundaries shapefile
        neighborhood_boundaries_path = '/home/jupyter/property_pilot/data/neighborhood_boundaries/geo_export_825d7df4-a9cd-4cef-b3d7-2ec1adc30204.shp'
        # neighborhood description file
        neighborhood_info_path = '/home/jupyter/property_pilot/data/neighborhood_info/neighborhood_info_final.csv'

        # load the neighborhood boundaries and description file
        neighborhoods = load_neighborhood_boundaries(neighborhood_boundaries_path)
        neighborhoods_info = load_neighborhood_info(neighborhood_info_path)

        # get neighborhood details of the properties
        top_properties = fetch_neighborhood(top_properties, neighborhoods)
        top_properties = fetch_neighborhood_info(top_properties, neighborhoods_info)

        # create property map
        property_map = create_property_map(top_properties)
        
        # create property_restaurant map
        property_restaurant_map = create_property_restaurant_map(top_properties)

        # change format
        fields_desc = fields + ['description', 'resoFacts', 'schools', 'neighborhood', 'neighborhood_description', 'restaurant_1', 'restaurant_2']
        property_info = format_properties(top_properties, fields_desc)

        # prompt for final output from Gemini
        prompt_final = generate_prompt_property(instruction_prompt_property, user_query, property_info, str(fields))

        # get final response from Gemini
        print(get_chat_response(chat, prompt_final))
        display(property_restaurant_map)

        return top_properties

    elif response_int == 2:
        # print('The query is about restaurant. I will answer by using Yelp API Filter')
        yelp_prompt = generate_prompt_yelp_advisor(yelp_advisor_instruction, user_query, categories_string)
        
        # get yelp filter string
        yelp_filter_str = get_chat_response(chat, yelp_prompt)
        
        # get yelp filter in json format
        yelp_filter = extract_json_to_dict(yelp_filter_str)
        
        # get response from yelp api
        yelp_advisor_results = yelp_advisor(yelp_api_key, yelp_filter)
        
        # create yelp restaurant map
        restaurant_map = create_yelp_restaurant_map(yelp_advisor_results)
        
        # generate yelp output prompt
        yelp_output_prompt = final_output_yelp_advisor(yelp_output_instructions, user_query, yelp_advisor_results)
        
        # get final response from Gemini
        print(get_chat_response(chat, yelp_output_prompt))
        display(restaurant_map)
        
    elif response_int == 3:
        # print('The query is about neighborhood. I will answer by using RAG')
        context = get_context(question = user_query, vector_store=vector_store, num_docs=10)
        prompt_rag_neighborhood = generate_prompt_rag_neighborhood(instruction_rag_neighborhood, context, user_query)
        print(get_chat_response(chat, prompt_rag_neighborhood))
        # return context
    else:
        print(get_chat_response(chat, user_query))

## Use case 1
### property search - detail info

In [124]:
# For initial query, use pp_chatbot, not get_chat_response
user_query = "I am looking to purchase a house in Chicago. I would like to have a minimum 2 bedrooms and 2 bathrooms. I want to pay between $700,000 and $900,000."
top_properties = pp_chatbot(user_query)

## Here are 3 properties that fit your criteria:

**1. Single-family home in Kenwood:**

* **Address:** 4922P S Cornell Ave, Chicago, IL 60615
* **Price:** $825,000
* **Bedrooms:** 3
* **Bathrooms:** 4
* **Details:** This spacious single-family home has 3 bedrooms and 4 bathrooms. It features a sunken family room, a balcony, and a finished basement. The kitchen has plenty of counter space and a walk-in pantry. There is a two-car attached garage and guest parking available. The home is located near public transportation and the lakefront.
* **Photos:** https://photos.zillowstatic.com/fp/6c85ab0c974c11627d33d83701c78839-p_e.jpg
* **More information:** https://www.zillow.com/homedetails/4922P-S-Cornell-Ave-Chicago-IL-60615/350418760_zpid/

**2. Modern condo in Wicker Park:**

* **Address:** 1332 N Leavitt St, Chicago, IL 60622
* **Price:** $725,000
* **Bedrooms:** 3
* **Bathrooms:** 3
* **Details:** This modern condo has 3 bedrooms and 3 bathrooms. It features high ceilings, a spacious ki

In [125]:
top_properties

[{'propertyType': 'SINGLE_FAMILY',
  'address': '4922P S Cornell Ave, Chicago, IL 60615',
  'price': 825000,
  'bedrooms': 3,
  'bathrooms': 4,
  'detailUrl': 'https://www.zillow.com/homedetails/4922P-S-Cornell-Ave-Chicago-IL-60615/350418760_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/6c85ab0c974c11627d33d83701c78839-p_e.jpg',
  'longitude': -87.5879,
  'latitude': 41.806026,
  'description': "Unit P, Welcome to one of Cornell Square's highly desired EXTRA wide single family home. Step into this stunning home's main level featuring a charming foyer and steps leading down to the sunken family room, perfect for cozy family movie nights or easily convertible to a 4th bedroom if needed. The foyer and family room boast rare heated flooring and built-in surround sound speakers, with additional speakers installed in every room throughout the home. A convenient half bath is located on this level, just before the two-car attached garage. Ascending to the 2nd floor, you'll experience

In [137]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Can you tell me more about the house in Kenwood?"
print(get_chat_response(chat, user_query))

## Unveiling the Details of the Kenwood Home: A Comprehensive Look

The single-family home in Kenwood presents a perfect blend of modern elegance and cozy comfort, providing a welcoming retreat for residents. Let's delve into the details of this inviting abode:

**Layout and Amenities:**

* **Spaciousness:** The home boasts three bedrooms and four bathrooms, offering ample space for families or individuals who desire a spacious living environment.
* **Sunken Family Room:** This unique feature adds a touch of whimsy and intimacy to the living area, creating a cozy space for movie nights or family gatherings.
* **Balcony:** Enjoy the fresh air and stunning city views from your private balcony, perfect for relaxing evenings or morning coffee breaks.
* **Finished Basement:** The finished basement adds valuable living space to the home, ideal for a home theater, playroom, or guest suite.
* **Kitchen Delight:** The chef's dream kitchen features abundant maple cabinetry, granite countertops, 

In [132]:
user_query = "Which neighborhood is the first property located in? Tell me more about that neighborhood."
print(get_chat_response(chat, user_query))

## Kenwood: A Haven of History and Tranquility

The first property you mentioned is located in the Kenwood neighborhood, a beautiful and historic enclave nestled alongside Lake Michigan in Chicago. Renowned for its architectural diversity and serene atmosphere, Kenwood presents a unique blend of urban convenience and suburban charm. Let's delve deeper into the essence of this captivating neighborhood:

**A Tapestry of Architectural Gems:**

Kenwood is a treasure trove for architecture enthusiasts, with an array of architectural styles gracing its streets. From stately mansions reminiscent of a bygone era to contemporary homes boasting modern designs, the neighborhood showcases a rich tapestry of architectural expressions. Among the notable structures are the impressive Powhatan Apartments, a testament to Art Deco elegance, and the Hyde Park–Kenwood Historic District, where history unfolds through its diverse collection of architectural marvels.

**A Legacy of Prominent Residents:**

Ke

In [133]:
user_query = "What are some good schools nearby the first property?"
print(get_chat_response(chat, user_query))

## Exploring Educational Opportunities near the First Property in Kenwood

The neighborhood surrounding the first property is fortunate to have a variety of reputable educational options catering to different age groups and educational philosophies. Let's delve into some of the notable schools in close proximity:

**Elementary Schools:**

* **Shoesmith Elementary School:** Located just 0.3 miles from the property, Shoesmith Elementary School boasts an impressive GreatSchools rating of 5 stars. This high-performing public school is renowned for its nurturing learning environment and dedicated teachers who strive to ignite a passion for learning in their young students.

**Middle Schools:**

* **Kenwood Academy High School:** Nestled within a walking distance of 0.2 miles, Kenwood Academy High School is a public school consistently ranked among the top middle schools in the city. With its strong academic programs, diverse extracurricular activities, and emphasis on fostering well-rounded

In [134]:
user_query = "I am interested in the first property. Should I buy it or rent it ?"
print(get_chat_response(chat, user_query))

## Assessing the Pros and Cons of Buying vs. Renting the First Property

Deciding between buying and renting the first property involves careful consideration of numerous factors that align with your individual needs, financial circumstances, and future goals. Here's a breakdown of the potential advantages and disadvantages of both options to help guide your decision-making process:

**Buying the First Property:**

**Pros:**

* **Building Equity:** As you pay down the mortgage, you accumulate equity in the property, increasing its value over time.
* **Property Appreciation:** Over the long term, real estate values often appreciate, which could result in a significant financial gain when you sell the property.
* **Sense of Ownership and Control:** Homeownership provides you with a sense of stability and control over your living environment. You can customize the property to your liking and make long-term improvements without constraints.
* **Tax Benefits:** Homeowners enjoy certain tax 

In [135]:
user_query = "What restaurants are nearby the first property? Please provide me with the yelp link"
print(get_chat_response(chat, user_query))

## Culinary Delights Await near the First Property

The first property in Kenwood is fortunate to be surrounded by an array of diverse and delectable culinary experiences, offering a delightful range of flavors, cuisines, and ambiances. Here are some of the notable restaurants you'll find just a stone's throw away, along with their corresponding Yelp links for further exploration:

**Capital One Café:**

* Distance from property: Approximately 1.2 miles
* Cuisine: Cafes
* Yelp link: https://www.yelp.com/biz/capital-one-café-chicago-6?adjust_creative=6Rl6qXMI6EN3VkijeNTf0A
* This charming café, located within 15 minutes walking distance, offers a cozy atmosphere perfect for enjoying a cup of freshly brewed coffee, indulging in a delicious pastry, or savoring a light lunch.

**Gorée Cuisine:**

* Distance from property: Approximately 1.5 miles
* Cuisine: Senegalese, Caterers
* Yelp link: https://www.yelp.com/biz/gor%C3%A9e-cuisine-chicago?adjust_creative=6Rl6qXMI6EN3VkijeNTf0A
* For a ta

In [136]:
top_properties

[{'propertyType': 'SINGLE_FAMILY',
  'address': '4922P S Cornell Ave, Chicago, IL 60615',
  'price': 825000,
  'bedrooms': 3,
  'bathrooms': 4,
  'detailUrl': 'https://www.zillow.com/homedetails/4922P-S-Cornell-Ave-Chicago-IL-60615/350418760_zpid/',
  'imgSrc': 'https://photos.zillowstatic.com/fp/6c85ab0c974c11627d33d83701c78839-p_e.jpg',
  'longitude': -87.5879,
  'latitude': 41.806026,
  'description': "Unit P, Welcome to one of Cornell Square's highly desired EXTRA wide single family home. Step into this stunning home's main level featuring a charming foyer and steps leading down to the sunken family room, perfect for cozy family movie nights or easily convertible to a 4th bedroom if needed. The foyer and family room boast rare heated flooring and built-in surround sound speakers, with additional speakers installed in every room throughout the home. A convenient half bath is located on this level, just before the two-car attached garage. Ascending to the 2nd floor, you'll experience

## Use case 2
### Initial query: Restaurant info

In [29]:
chat = model.start_chat()
# Test chat
# prompt = "Hello."
# print(get_chat_response(chat, prompt))

In [33]:
# For initial query, use pp_chatbot, not get_chat_response
user_query = "What are the top rated Indian restaurants on the north side of chicago?"
pp_chatbot(user_query)

Absolutely! Based on your query, here are some of the top-rated Indian restaurants on the north side of Chicago:

1. **Art of Indian Cuisine (4.9 stars):** This highly-rated gem offers authentic Indian dishes with a focus on fresh, seasonal ingredients. Reviewers rave about the flavorful curries, delectable tandoori dishes, and friendly service.

2. **Bhoomi Modern Indian Grill (4.7 stars):** This modern eatery boasts a sophisticated atmosphere and a menu featuring innovative takes on classic Indian cuisine. Enjoy plates like black truffle naan, lamb chops, and creative vegetarian options.

3. **Krishna Lunch (4.7 stars):** This vegetarian lunch spot is a haven for those seeking flavorful and affordable Indian fare. The menu changes daily, offering a variety of curries, rice dishes, and fresh salads.

4. **Apna Indian Grill (4.6 stars):** This casual eatery offers a wide range of classic Indian dishes, from curries and biryanis to tandoori specialties. The portions are generous, and th

In [34]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Which of these restaurants are budget-friendly?"
print(get_chat_response(chat, user_query))

## Budget-Friendly Indian Restaurants on the North Side

While all the restaurants I recommended have reasonable prices for their quality, here's a breakdown based on online information and reviews:

**Most Budget-Friendly:**

* **Krishna Lunch:** This vegetarian spot is known for its incredibly affordable lunch buffets, starting at around $10. You can also get individual dishes for under $10.
* **Apna Indian Grill:** This casual eatery offers generous portions at reasonable prices. Most dishes are priced between $10-$15.

**Moderately Priced:**

* **Art of Indian Cuisine:** While offering high-quality ingredients, this restaurant has a slightly higher price range. Most entrees fall between $15-$25.
* **Bhoomi Modern Indian Grill:** This modern restaurant has a contemporary atmosphere and slightly higher price point than the others. Entrees typically range from $20-$35.
* **Indian Paradise:** This family-owned restaurant offers a good value for its authentic North Indian dishes. Most e

In [36]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Could you provide me a link to Bhoomi Modern Indian Grill's menu?"
print(get_chat_response(chat, user_query))

## Bhoomi Modern Indian Grill Menu

Here's the link to Bhoomi Modern Indian Grill's menu: https://www.eatbhoomi.com/menu

Please note that online menus may not always be completely up-to-date, so it's always a good idea to call the restaurant directly for the most accurate information.





## Use case 3
### Initial query: Neighborhood info

In [39]:
chat = model.start_chat()

In [40]:
# For initial query, use pp_chatbot, not get_chat_response
user_query = "What is the Hyde Park in Chicago like?"
pp_chatbot(user_query)

## Hyde Park in Chicago: A Detailed Look

Hyde Park is a vibrant neighborhood on the south side of Chicago, offering a unique blend of history, culture, and natural beauty. Here's a breakdown of its key features:

**Things to Do**:

* **Museums & Culture:** Hyde Park is home to the renowned University of Chicago and its associated museums, including the Oriental Institute Museum and the Smart Museum of Art. The neighborhood also boasts the iconic Museum of Science and Industry, a family-friendly destination with interactive exhibits and educational programs.
* **Green Spaces:** Relax and unwind in the expansive Jackson Park, offering stunning views of Lake Michigan, walking trails, and the beautiful landscaped gardens. The park also houses the Museum of Science and Industry and the Osaka Garden, a traditional Japanese garden.
* **Diverse Dining:** Hyde Park caters to a variety of palates with its wide selection of restaurants. From casual eateries and cafes to upscale dining options, t

In [41]:
# For additional question, use get_chat_response, not pp_chatbot
user_query = "Tell me more about the neighborhood?"
print(get_chat_response(chat, user_query))

## Delving Deeper into Hyde Park:

While I've covered the main highlights of Hyde Park, there's always more to explore in this dynamic neighborhood. Here are some additional details:

**History:**

* Founded in 1853, Hyde Park boasts a rich history. Its development as a planned community was envisioned by Paul Cornell, who aimed to create a picturesque suburb with spacious lots, tree-lined streets, and a commitment to education and culture.
* The arrival of the University of Chicago in 1892 significantly shaped the neighborhood. Its iconic campus architecture and academic reputation attracted intellectuals, artists, and scholars, contributing to Hyde Park's vibrant intellectual atmosphere.
* Throughout the 20th century, Hyde Park played a pivotal role in social movements like the Civil Rights movement. Its diverse population and progressive values fueled activism and social change initiatives.

**Architecture:**

* Hyde Park showcases a diverse array of architectural styles, reflecting